# Case study of uniaxial tensile tests

This is an example of how to use Paramaterial to process a dataset of uniaxial tensile test measurements.
The data should be formatted as a set of csv files containing measurements for each test, and a single excel spreadsheet containing the metadata for the dataset.
There should be a single row in the excel spreadsheet for each csv file.

The analysis takes 4 stages:
- Data preparation
- Data processing
- Model fitting
- Test report generation

## Modelling
- Make representative curves
- Fit models
- Fit equations for model parameters

In [1]:
from typing import Callable

import numpy as np
from matplotlib import pyplot as plt

import paramaterial as pam
from paramaterial import ModelSet
from paramaterial.models import ramberg
from paramaterial.plug import DataSet, DataItem

Start with processed data.

In [2]:
processed_ds = DataSet('data/02 processed data', 'info/02 processed info.xlsx')

Setup plotting.

In [3]:
styler = pam.plotting.Styler(
    color_by='temperature', cmap='plasma', color_by_label='(°C)', color_norm=plt.Normalize(20, 320),
    plot_kwargs=dict(grid=True))
styler.style_to(processed_ds.sort_by(['temperature', 'lot']))


def ds_plot(ds: DataSet, **kwargs):
    return pam.plotting.dataset_plot(ds, styler=styler, **kwargs)


subplot_cfg = dict(
    shape=(3, 3), sharex='all', sharey='all', hspace=0.2,
    rows_by='lot', row_vals=[[a] for a in 'ABCDEFGHI'],
    cols_by='lot', col_vals=[[a] for a in 'ABCDEFGHI'],
    plot_titles=[f'Lot {a}' for a in 'ABCDEFGHI']
)


def ds_subplots(ds: DataSet, **kwargs):
    return pam.plotting.dataset_subplots(ds=ds, styler=styler, plot_legend=False, **subplot_cfg, **kwargs)


def subplot_wrapper(ds: DataSet, plot_func: Callable[[DataItem], DataItem], **plot_func_kwargs):
    return pam.plotting.subplot_wrapper(ds=ds, plot_func=plot_func, **subplot_cfg, **plot_func_kwargs)


stress_strain_labels = dict(x='Strain', y='Stress_MPa', ylabel='Stress (MPa)')

FileNotFoundError: [Errno 2] No such file or directory: 'data/02 processed data/test_ID_001.csv'

### Make representative curves
There is clear temperature dependence.
We are also interested in differences between lots.
We therefore make representative curves:
- Grouped by temperature
- Grouped by temperature and lot

Make representative curves grouped by temperature.

In [ ]:
pam.processing.make_representative_data(ds=processed_ds,
                                        data_dir='data/03 temp repr data', info_path='info/03 temp repr info.xlsx',
                                        repr_col='Stress_MPa', repr_by_cols=['temperature'],
                                        interp_by='Strain', interp_end='min_of_maxes', interp_res=1000)
temp_repr_ds = DataSet('data/03 temp repr data', 'info/03 temp repr info.xlsx', test_id_key='repr id')
temp_repr_ds

Plot representative curves grouped by temperature.

In [ ]:
repr_stress_strain_labels = dict(x='interp_Strain', y='mean_Stress_MPa',
                                 fill_between=('min_Stress_MPa', 'max_Stress_MPa'),
                                 xlabel='Strain', ylabel='Stress (MPa)')  # todo: make error bar plots
ds_plot(temp_repr_ds, **repr_stress_strain_labels);

Make representative curves grouped by temperature and lot.

Plot representative curves grouped by temperature and lot.

### Fit models
- Trim to 0.01 Strain
- Fit models to temperature grouped curves
- Fit models to temperature and lot grouped curves

In [ ]:
def trim_for_fitting(di):
    di.data = di.data[di.data['interp_Strain'] <= 0.01]
    return di


temp_repr_ds = temp_repr_ds.apply(trim_for_fitting)

### Fit models to small-strain representative data

In [ ]:
# ms = ModelSet(ramberg, ['E', 's_y', 'C', 'n'], bounds=[(35e3, 90e3), (1., 280.), (30, 220.), (0.01, 0.8)], scipy_func='basinhopping')
ms = ModelSet(ramberg, param_names = ['C', 'n'], var_names=['mean_E', 'mean_s_y'],
              bounds=[(30, 220.), (0.01, 0.8)], scipy_func='basinhopping')
ms.fit(temp_repr_ds, 'interp_Strain', 'mean_Stress_MPa', sample_size=40)
temp_ramberg_ds = ms.predict()

In [ ]:
print(temp_ramberg_ds.info_table)
ms.params_table

In [ ]:
ax = ds_plot(temp_repr_ds, x='interp_Strain', y='mean_Stress_MPa', alpha=0.3, lw=0, marker='o', mfc='none',figsize=(10,8))
ds_plot(temp_ramberg_ds, x='x', y='y', ls='--', ax=ax);